<a href="https://colab.research.google.com/github/tozdo/2022-hse-AML/blob/main/HSE_AML_HW_1_Zdorova.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.optimize as opt
import sklearn.linear_model
import sklearn.model_selection
from sklearn import metrics

According to the documentation (https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_20newsgroups.html), when loading 20newsgroup dataset, by default it loads 'train' data. But then we are supposed to divide this (using KFold) to train and test sets. So i decided to set parameter 'subset' to 'all' to load all the data and then divide it myself.

In [ ]:
from sklearn.datasets import fetch_20newsgroups
data = fetch_20newsgroups(subset='all', categories=['alt.atheism', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey'])

Also I have decided to use only some of the categories in the dataset because it is less data and it computes faster.

Use HashingVectorizer to encode the text into sparse features:

In [ ]:
from sklearn.feature_extraction.text import HashingVectorizer

In [ ]:

vectorizer = HashingVectorizer(n_features=1000000, binary=True, norm=None)
X = vectorizer.fit_transform(data['data'])

I have used 1 million as a n_features, because I think it is enough for English dictionary, and also it allows me train the model. Because in other case RAM in Google Colab is full.

In [ ]:
y = np.array(data['target'])

In [ ]:
print(X.shape, len(y))

(4778, 1000000) 4778


Use the K-Fold cross-validation to split the dataset into training and test parts:

In [ ]:
xval = sklearn.model_selection.KFold(n_splits=3, shuffle=True, random_state=7)
for train, test in xval.split(X):
    X_train, X_test = X[train], X[test]
    y_train, y_test = y[train], y[test]

Experiment with different models (L1, L2, ...)

Model L1

In [ ]:
classification_model = sklearn.linear_model.SGDClassifier(loss='log', penalty='l1');
regression_model = sklearn.linear_model.SGDRegressor(penalty='l1');

In [ ]:
classification_model.fit(X_train, y_train)
classification_model_predictions = classification_model.predict(X_test)

I will use F1 score  metric. for classification model and R2 metric for regression model, because I need to decide which model worked best, and I think these metrics are the most common.

In [ ]:
metrics.f1_score(classification_model_predictions, y_test, average='macro')

0.9466292533102616

In [ ]:
regression_model.fit(X_train, y_train)
regression_model_pred = regression_model.predict(X_test)

In [ ]:
metrics.r2_score(regression_model_pred, y_test)

0.766170172710806

Model L2

In [ ]:
classification_model_2 = sklearn.linear_model.SGDClassifier(loss='log', penalty='l2');
regression_model_2 = sklearn.linear_model.SGDRegressor(penalty='l2');

In [ ]:
classification_model_2.fit(X_train, y_train)
classification_model_2_pred = classification_model_2.predict(X_test)

In [ ]:
metrics.f1_score(classification_model_2_pred, y_test, average='macro')

0.962271487376508

In [ ]:
regression_model_2.fit(X_train, y_train)
regression_model_2_pred = regression_model_2.predict(X_test)

In [ ]:
metrics.r2_score(regression_model_2_pred, y_test)

0.7733005189512924

Model L1+L2

In [ ]:
regression_model_3 = sklearn.linear_model.SGDRegressor(penalty='elasticnet')

In [ ]:
regression_model_3.fit(X_train, y_train)
regression_model_3_pred = regression_model_3.predict(X_test)

In [ ]:
metrics.r2_score(regression_model_3_pred, y_test)

0.7736471560762849


What model worked best?

According to metrics result, L2 model worked the best. I guess this is because we do not have useless features and every one is bettering the perfomance of the model.